# Desafio Kinea


## Importando e definindo constantes globais

In [1]:
import pandas as pd
from typing import Dict, List
import csv
from dateutil.relativedelta import relativedelta

In [2]:
from ambima_connect import AmbimaConnect

In [3]:
import logging
logging.level=logging.INFO

In [4]:
from datetime import date, datetime
from cal import AmbimaCalendar

## Funções básicas

In [5]:
def dias_uteis(start_date, end_date) -> int:
  """O formato de data é date(AAAA, MM, DD), como date(2022, 2, 4) para 4 de fev de 2022"""
  interval = AmbimaCalendar().get_working_days_delta(start_date, end_date)+1
  return interval

In [6]:
def check_error(acquired_value, expected_value):
  err = abs((acquired_value-expected_value)/expected_value)*100
  err_str = f'{err:0.3}%'
  return err_str

## Interface com API Ambima

In [7]:
# data_res = get_data(client_id, access_token, adress_vna)
data_list = AmbimaConnect('titulos publicos').content
data_vna = AmbimaConnect('vna').content

In [8]:
def save_to_csv(filedata:List[Dict], filename:str):
  """Take a list of dicts and save as an .csv with the given name"""
  keys = filedata[0].keys()
  with open('exports\\'+filename, 'w', newline='') as output_file:
      dict_writer = csv.DictWriter(output_file, keys)
      dict_writer.writeheader()
      dict_writer.writerows(filedata)

In [9]:
def clean_mercado_secundario(list_to_clean:List[Dict]):
  keep_keys = ['tipo_titulo','data_referencia','data_vencimento','taxa_indicativa','pu']
  res = [{key : val for key, val in sub.items() if key in keep_keys} for sub in list_to_clean]
  return res

In [10]:
clean_mercado = clean_mercado_secundario(data_list)

## Cálculo de PUs

In [11]:
def get_dias_cupom(start_date, end_date):
  """Retorna uma listagem de cupons entre duas datas, indo da data final até a inicial, de 6 em 6 meses"""
  pgto_date_list = list()
  pgto_date = end_date
  # Gera da lista de datas a partir da data final, em incrementos de 6 meses
  while pgto_date > start_date:
    pgto_date_list.append(pgto_date)
    pgto_date -= relativedelta(months=6)
  # Inverte a ordem da lista para crescente
  pgto_date_list=pgto_date_list[::-1]
  
  return pgto_date_list

In [30]:
[x for x in clean_mercado if x['tipo_titulo']=='LFT'][0]

{'tipo_titulo': 'LFT',
 'data_vencimento': '2018-03-01',
 'data_referencia': '2017-10-20',
 'taxa_indicativa': 0,
 'pu': 9162.202647}

LTN

In [21]:
class LTN:
  def __init__(self, data_venc, taxa, data_ref):
    self.data_venc = data_venc
    self.taxa = taxa
    self.data_ref = data_ref

    self.duration = dias_uteis(data_ref, data_venc)
    self.pm = (data_venc - data_ref).days
    self.pu = 1000/((1+taxa)**(self.duration/252))

In [17]:
LTN(date(2018,1,1), 7.2843/100, date(2017,10,20)).pu

986.6964968762362

NTN-F

In [22]:
class NTNF():
  def __init__(self, data_venc, taxa, data_ref):
    self.data_venc = data_venc
    self.taxa = taxa
    self.data_ref = data_ref
    
    lista_datas = get_dias_cupom(data_ref, data_venc)
    self.fluxo_pgto = pd.DataFrame(lista_datas, columns=['data_pgto'])
    self.fluxo_pgto['cupom_semestral'] = [(1.1**(1/2)-1)*1000]*len(lista_datas)
    self.fluxo_pgto['cupom_semestral'].iloc[-1] = self.fluxo_pgto['cupom_semestral'].iloc[-1] + 1000
    self.fluxo_pgto['dias_uteis'] = self.fluxo_pgto['data_pgto'].apply(lambda x: dias_uteis(data_ref, x))
    self.fluxo_pgto['dias_corridos'] = self.fluxo_pgto['data_pgto'].apply(lambda x: (x - data_ref).days)
    self.fluxo_pgto['vp_pgto'] = self.fluxo_pgto.apply(lambda x: (x['cupom_semestral'])/((1+taxa)**(x['dias_uteis']/252)), axis=1)

    self.pu = self.fluxo_pgto['vp_pgto'].sum()
    self.duration = (self.fluxo_pgto['vp_pgto']*self.fluxo_pgto['dias_uteis']).sum()/self.pu
    self.pm = (self.fluxo_pgto['cupom_semestral']*self.fluxo_pgto['dias_corridos']).sum()/self.fluxo_pgto['cupom_semestral'].sum()

In [20]:
NTNF(date(2018,1,1), 7.3002/100, date(2017,10,20)).pu

C:\Users\HP\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


1034.826805594595

NTN-B

In [23]:
class NTNB():
  def __init__(self, data_venc, taxa, data_ref):
    self.data_venc = data_venc
    self.taxa = taxa
    self.data_ref = data_ref
  # d15_base = data_liqui.replace(day=15)
  # d15_proj = d15_base + relativedelta(months=1)
  
  # VNA deveria puxar dados de um banco, porém ainda não existe esse banco e o API tem só uma data em sandbox
    self.vna = [x['vna'] for x in data_vna[0]['titulos'] if x['codigo_selic']=='760100'][0]

    lista_datas = get_dias_cupom(data_ref, data_venc)
    self.fluxo_pgto = pd.DataFrame(lista_datas, columns=['data_pgto'])
    self.fluxo_pgto['fluxo'] = [(1.06**(1/2)-1)]*len(lista_datas)
    self.fluxo_pgto['fluxo'].iloc[-1] = self.fluxo_pgto['fluxo'].iloc[-1] + 1
    self.fluxo_pgto['dias_uteis'] = self.fluxo_pgto['data_pgto'].apply(lambda x: dias_uteis(data_ref, x))
    self.fluxo_pgto['dias_corridos'] = self.fluxo_pgto['data_pgto'].apply(lambda x: (x - data_ref).days)
    self.fluxo_pgto['vf_fluxo_taxa'] = self.fluxo_pgto.apply(lambda x: ((1+taxa)**(x['dias_uteis']/252)), axis=1)
    self.fluxo_pgto['cotacao'] = self.fluxo_pgto.apply(lambda x: x['fluxo']/x['vf_fluxo_taxa'], axis=1)

    self.fluxo_pgto['cupom_semestral'] = self.fluxo_pgto['fluxo']*self.vna
    self.fluxo_pgto['vp_pgto'] = self.fluxo_pgto['cupom_semestral']/self.fluxo_pgto['vf_fluxo_taxa']

    self.pu = self.fluxo_pgto['cotacao'].sum()*self.vna
    self.duration = (self.fluxo_pgto['vp_pgto']*self.fluxo_pgto['dias_uteis']).sum()/self.fluxo_pgto['vp_pgto'].sum()
    self.pm = (self.fluxo_pgto['cupom_semestral']*self.fluxo_pgto['dias_corridos']).sum()/self.fluxo_pgto['cupom_semestral'].sum()

In [26]:
NTNB(date(2018,8,15), 2.55/100, date(2017,10,20)).pu

C:\Users\HP\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


3127.6391868892265

LFT

In [28]:
class LFT:
  def __init__(self, data_venc, taxa, data_ref):
    self.data_venc = data_venc
    self.taxa = taxa
    self.data_ref = data_ref
    
    self.duration = dias_uteis(data_ref, data_venc)
    self.pm = (data_venc - data_ref).days

    self.vna = [x['vna'] for x in data_vna[0]['titulos'] if x['codigo_selic']=='210100'][0]
    self.cotacao = (1+taxa)**(-self.duration/252)
    self.pu = self.vna*self.cotacao

In [31]:
LFT(date(2018,3,1),0,date(2017,10,20)).pu

9162.202647

## Cálculo Geral

In [32]:
for titulo in clean_mercado:
  data_ref=datetime.strptime(titulo['data_referencia'], "%Y-%m-%d").date()
  data_venc=datetime.strptime(titulo['data_vencimento'], "%Y-%m-%d").date()

  if titulo['tipo_titulo'] == 'LTN':
    calc = LTN(
      data_venc=data_venc,
      taxa=titulo['taxa_indicativa']/100,
      data_ref=data_ref,
      )

  elif titulo['tipo_titulo'] == 'NTN-B':
    calc = NTNB(
      data_venc=data_venc,
      taxa=titulo['taxa_indicativa']/100,
      data_ref=data_ref,
      )

  elif titulo['tipo_titulo'] == 'NTN-F':
    calc = NTNF(
      data_venc=data_venc,
      taxa=titulo['taxa_indicativa']/100,
      data_ref=data_ref,
      )

  elif titulo['tipo_titulo'] == 'LFT':
    calc = LFT(
      data_venc=data_venc,
      taxa=titulo['taxa_indicativa']/100,
      data_ref=data_ref,
      )

  titulo['pu_calc']=calc.pu
  titulo['pm_calc']=calc.pm
  titulo['duration_calc']=calc.duration

C:\Users\HP\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [35]:
t =[x for x in clean_mercado if x['tipo_titulo'] in ['NTN-F', 'NTN-B', 'LFT', 'LTN']]
print('Tipo, Data Venc, Real, Calc, Error')
for x in t: print(f"{x['tipo_titulo']}, {x['data_referencia']}, {x['pu']}, {x['pu_calc']}, {check_error(x['pu_calc'], x['pu'])}")

Tipo, Data Venc, Real, Calc, Error
LTN, 2017-10-20, 986.696496, 986.6964968762362, 8.88e-08%
NTN-F, 2017-10-20, 1034.826807, 1034.826805594595, 1.36e-07%
LFT, 2017-10-20, 9162.202647, 9162.202647, 0.0%
LTN, 2017-10-20, 970.982414, 970.9824147548106, 7.77e-08%
LTN, 2017-10-20, 954.999495, 954.9994953886635, 4.07e-08%
NTN-B, 2017-10-20, 3127.950824, 3127.6391868892265, 0.00996%
LFT, 2017-10-20, 9162.202647, 9162.202647, 0.0%
LTN, 2017-10-20, 938.119221, 937.8653345011708, 0.0271%
LTN, 2017-10-20, 920.958993, 920.958993577912, 6.28e-08%
NTN-F, 2017-10-20, 1060.800212, 1060.8002073544483, 4.38e-07%
LFT, 2017-10-20, 9162.001078, 9162.004281775411, 3.5e-05%
LTN, 2017-10-20, 902.67683, 902.4194136427137, 0.0285%
NTN-B, 2017-10-20, 3238.274966, 3237.9476706693135, 0.0101%
LTN, 2017-10-20, 883.262368, 883.0019750547143, 0.0295%
LTN, 2017-10-20, 861.87865, 861.6156307707371, 0.0305%
LTN, 2017-10-20, 842.202192, 842.2021922041295, 2.42e-08%
LFT, 2017-10-20, 9161.55213, 9161.558052944505, 6.47e-05